In [ ]:
from pyagena import *

In [ ]:
#Creating a model and its network

diet = Model()

diet.create_network("Hierarchical_Normal_Model_1")
net = diet.get_network("Hierarchical_Normal_Model_1")

In [ ]:
#Creating mean and variance nodes

net.create_node(id="mean", simulated=True)
net.set_node_expressions(node_id="mean", expressions=["Normal(0.0,100000.0)"])

net.create_node(id="variance", simulated=True)
net.set_node_expressions(node_id="variance", expressions=["Uniform(0.0,50.0)"])

In [ ]:
#Now we create the "common variance" and its "tau" parameter nodes

net.create_node(id="tau", simulated=True)
net.set_node_expressions(node_id="tau", expressions=["Gamma(0.001,1000.0)"])

net.create_node(id="common_var", name="common variance", simulated=True)
net.create_edge(child_id="common_var", parent_id="tau")
net.set_node_expressions(node_id="common_var", expressions=["Arithmetic(1.0/tau)"])

In [ ]:
#Creating a list of four mean nodes, "mean A", "mean B", "mean C", and "mean D"

mean_names = ["A", "B", "C", "D"]
means_list = []

for mn in mean_names:
    this_id = "mean" + mn
    this_name = "mean " + mn
    net.create_node(id=this_id, name=this_name)
    net.create_edge(child_id=this_id, parent_id="mean")
    net.create_edge(child_id=this_id, parent_id="variance")
    net.set_node_expressions(node_id=this_id, expressions=["Normal(mean,variance)"])
    means_list.append(this_id)

In [ ]:
# Defining the list of observations for the experiment nodes
# and creating the experiment nodes y11, y12, ..., y47, y48

observations = [[62, 60, 63, 59],
                [63, 67, 71, 64, 65, 66],
                [68, 66, 71, 67, 68, 68],
                [56, 62, 60, 61, 63, 64, 63, 59]]

for i, (obs, mn) in enumerate(zip(observations, means_list)):
    for j, ob in enumerate(obs):
        this_id = "y"+str(i)+str(j)
        net.create_node(id=this_id, simulated=True)
        net.create_edge(child_id=this_id, parent_id="common_var")
        net.create_edge(child_id=this_id, parent_id=mn)
        net.set_node_expressions(node_id=this_id, expressions=["Normal("+mn+",common_var)"])

In [ ]:
# Entering all the observations

for i, obs in enumerate(observations):
    for j, ob in enumerate(obs):
        this_node_id = "y" + str(i) + str(j)
        diet.enter_observation(network_id=net.id,
        node_id=this_node_id,
        value=ob)

In [ ]:
# To create a local .cmpx file
diet.save_to_file("./diet_model_example.cmpx")

In [ ]:
# Or sending it to local agena.ai developer API for calculation
local_api_calculate(diet)

In [ ]:
# Or sending it to agena.ai Cloud for calculation
user = login()
user.calculate(diet, "Case 1")